In [ ]:
save_figs = True

# Not enough chunks

In [ ]:
import pandas as pd
import nannyml as nml
reference, analysis, _ = nml.datasets.load_synthetic_binary_classification_dataset()
# metadata = nml.extract_metadata(reference, model_type='classification_binary', exclude_columns=['identifier'])
# metadata.target_column_name = 'work_home_actual'
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_number=8,
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
# Estimate on concatenated reference and analysis
est_perf = cbpe.estimate(pd.concat([reference, analysis], ignore_index=True))
# est_perf = cbpe.estimate(analysis)
est_perf.data#.iloc[3:5,:7]

In [ ]:
import pandas as pd
import nannyml as nml
reference, analysis, _ = nml.datasets.load_synthetic_binary_classification_dataset()

cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_period="Q",
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
est_perf = cbpe.estimate(analysis)

In [ ]:
cbpe = nml.CBPE(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    chunk_number=5,
    metrics=['roc_auc'],
    problem_type='classification_binary',
).fit(reference_data=reference)
est_perf = cbpe.estimate(analysis)

# Not enough observations in chunk

In [ ]:
calc = nml.PerformanceCalculator(
    y_pred_proba='y_pred_proba',
    y_pred='y_pred',
    y_true='work_home_actual',
    timestamp_column_name='timestamp',
    metrics=["roc_auc"],
    chunk_size=1,
    problem_type='classification_binary',
)

calc.fit(reference)
results = calc.calculate(reference)
results.data.head()

# Sampling error

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

sample_size = 100
dataset_size = 10_000
# random model
np.random.seed(23)
y_true = np.random.binomial(1, 0.5, dataset_size)
y_pred = np.random.binomial(1, 0.5, dataset_size)
accuracy_scores = []

for experiment in range(10_000):
    subset_indexes = np.random.choice(dataset_size, sample_size, replace=False) # get random indexes
    y_true_subset = y_true[subset_indexes]
    y_pred_subset = y_pred[subset_indexes]
    accuracy_scores.append(accuracy_score(y_true_subset, y_pred_subset))

plt.hist(accuracy_scores, bins=20, density=True)
plt.title("Accuracy of random classifier\n for randomly selected samples of 100 observations.");
if save_figs:
    plt.savefig("../_static/deep_dive_data_chunks_stability_of_accuracy.svg", format="svg")

In [ ]:
obs_level_accuracy = y_true == y_pred
np.mean(obs_level_accuracy), accuracy_score(y_true, y_pred)


In [ ]:
SEM_std = np.std(obs_level_accuracy)/np.sqrt(sample_size)
np.round(SEM_std, 3), np.round(np.std(accuracy_scores), 3)
